# Capstone Project

## Modeling Notebook - `PREPOST_dataset_COMBO` from 11/14/19

#### Importing Libraries

In [239]:
%matplotlib inline

# general libraries
import re
import string
import sys
import os
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# importing date libraries
import datetime as dt
import dateutil.parser as dparser

# scikit-learn libraries for preprocessing
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import PolynomialFeatures
from sklearn.impute import SimpleImputer

# scikit-learn libraries for constructing pipelines
from sklearn.pipeline import FeatureUnion, Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import FunctionTransformer
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.base import clone

# scikit-learn libraries for clustering and dimensionality reduction
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_samples, silhouette_score
from sklearn.cluster import DBSCAN
from scipy.cluster.hierarchy import dendrogram, linkage, fcluster
from sklearn.mixture import GaussianMixture

# scikit-learn libraries for evaluation
from sklearn import metrics
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc

# scikit-learn libraries for feature selection
from sklearn.feature_selection import VarianceThreshold
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2, f_classif
from sklearn.feature_selection import SelectPercentile
from sklearn.feature_selection import RFECV

# scikit-learn libraries for learning
from sklearn.model_selection import KFold, StratifiedKFold, GridSearchCV, cross_validate, cross_val_score
from sklearn.model_selection import validation_curve
from sklearn.model_selection import cross_val_predict
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier

# saving models
import pickle
from sklearn.externals import joblib

# setting pandas display options
pd.set_option("display.max_columns", 999)
pd.set_option("display.max_rows", 10000)
pd.set_option('display.max_colwidth', 100)
pd.set_option('precision', 5)
pd.options.mode.chained_assignment = None

#### Directory/File Structure

In [240]:
sys.version

'3.6.9 |Anaconda custom (x86_64)| (default, Jul 30 2019, 13:42:17) \n[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]'

In [241]:
print ('Running pandas version:', pd.__version__)
print ('Running numpy version:', np.__version__)
print ('Running sklearn version:', sklearn.__version__)

Running pandas version: 0.25.2
Running numpy version: 1.17.2
Running sklearn version: 0.21.3


In [242]:
os.getcwd()

'/Users/Shailesh/Desktop/MIDS/Capstone/w210CapstoneProject/data'

In [243]:
os.chdir('../data')

In [244]:
sorted(os.listdir())

['.DS_Store',
 'Capstone_Fall_Shannon_Sept2019_request.csv',
 'POSTOP_ALL_col_names_11_9.pkl',
 'POSTOP_TREE_ALL_col_names_11_9.pkl',
 'POSTOP_categorical_TREE_col_names_11_14.pkl',
 'POSTOP_categorical_TREE_col_names_11_9.pkl',
 'POSTOP_categorical_TREE_col_names_12_1.pkl',
 'POSTOP_categorical_col_names_11_14.pkl',
 'POSTOP_categorical_col_names_11_9.pkl',
 'POSTOP_categorical_col_names_12_1.pkl',
 'POSTOP_numerical_col_names_11_14.pkl',
 'POSTOP_numerical_col_names_11_9.pkl',
 'POSTOP_numerical_col_names_12_1.pkl',
 'PREOP_ALL_col_names_11_9.pkl',
 'PREOP_TREE_ALL_col_names_11_9.pkl',
 'PREOP_categorical_TREE_col_names_11_14.pkl',
 'PREOP_categorical_TREE_col_names_11_9.pkl',
 'PREOP_categorical_TREE_col_names_12_1.pkl',
 'PREOP_categorical_col_names_11_14.pkl',
 'PREOP_categorical_col_names_11_9.pkl',
 'PREOP_categorical_col_names_12_1.pkl',
 'PREOP_dataset_10_24.pkl',
 'PREOP_dataset_TREE_10_24.pkl',
 'PREOP_dataset_TREE_cabg.pkl',
 'PREOP_dataset_TREE_cabvalve.pkl',
 'PREOP_datas

## Preprocessing Unprocessed Dataset

### Loading Dataset

#### `PREPOST_dataset_COMBO_11_14`

In [245]:
data = pd.read_pickle('../data/PREPOST_dataset_COMBO_12_1.pkl')

In [246]:
data.head(2)

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,surgdt_month,surgdt_DayOfWeek,surgdt_PartOfMonth,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,numdisv,anginalclass,classnyh,vdinsufm,vdinsuft,incidencREOP,status,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%,cvdcarsten,cvdstenrt,cvdstenlft,recordId,predstro,cnstrokp,cnstrokttia,cncomaenceph,strokeBin,strokeBin2,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn,aortoccl,asmtaodx,cathbasassistind,cathbasassistwhen,cpbutil,cperftyp,ecmoind,ecmowhen,emergrsn,iabpind,iabpwhen,unplproc,urgntrsn
0,54,180.0,117.0,36.11111,43.0,0.9,3.8,7.2,6.5,47.0,42.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,7,4,1,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1,0.0,0.0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0,1.0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,3,3,0,4,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0.014,2,0,2,0,0,NaN,0.0,0.0,NaN,NaN,NaN,NaN,NaN,28.0,29.0,150.0,1.2,NaN,NaN,108.0,1.0,0.0,0.0,0.0,0.0,

In [247]:
data.shape

(42740, 254)

In [248]:
columns = data.columns

In [249]:
data= pd.read_csv('../dataWithProcedureType.csv')

In [250]:
data.shape

(42740, 263)

In [251]:
newColumns = data.columns
for col in newColumns:
    if col not in columns and col != 'procTyp':
        print(col)
        data.drop(col, axis=1, inplace=True)

mtopd
mtcause_Cardiac
mtcause_Infection
mtcause_Neuro
mtcause_Pulmonary
mtcause_Renal
mtcause_Vascular
mtcause


In [252]:
data.shape

(42740, 255)

In [253]:
data.head(2)

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,surgdt_month,surgdt_DayOfWeek,surgdt_PartOfMonth,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,numdisv,anginalclass,classnyh,vdinsufm,vdinsuft,incidencREOP,status,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%,cvdcarsten,cvdstenrt,cvdstenlft,recordId,predstro,cnstrokp,cnstrokttia,cncomaenceph,strokeBin,strokeBin2,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn,aortoccl,asmtaodx,cathbasassistind,cathbasassistwhen,cpbutil,cperftyp,ecmoind,ecmowhen,emergrsn,iabpind,iabpwhen,unplproc,urgntrsn,procTyp
0,54,180.0,117.0,36.11111,43.0,0.9,3.8,7.2,6.5,47.0,42.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,1,0,7,4,1,1,1,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,1,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,3,3,0,4,2,0,0,0,0,0,0,0,0,0,0,0,0,1,0.014,2,0,2,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,28.0,29.0,150.0,1.2,NaN,NaN,108.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,1,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,

In [254]:
cabg = data.loc[data['procTyp'] == 1]
valve = data.loc[data['procTyp'] == 2]
cabgValve = data.loc[data['procTyp'] == 3]

In [255]:
cabg.shape

(28945, 255)

In [256]:
valve.shape

(8500, 255)

In [257]:
cabgValve.shape

(5295, 255)

In [258]:
origData = data

In [259]:
data = cabg

### Dropping Irrelevant Columns

- `recordId`
- `cnstrokp`
- `cnstrokttia`
- `cncomaenceph`
- `strokeBin` as we are going to use the more inclusive `strokeBin2` as our outcome variable

#### Relative to prior versions, we are also going to retain `predstro` that will be used to compare our models versus the `STS` Model

In [260]:
cols_to_drop = ['recordId',
                'cnstrokp',
                'cnstrokttia',
                'cncomaenceph',
                'strokeBin',
               'procTyp']

In [261]:
len(cols_to_drop)

6

- dropping columns

In [262]:
data = data.drop(cols_to_drop, axis=1)

In [263]:
data.shape

(28945, 249)

- resetting `DataFrame` `index` 

In [264]:
data = data.reset_index(drop=True)

In [265]:
data.head()

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,surgdt_month,surgdt_DayOfWeek,surgdt_PartOfMonth,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,numdisv,anginalclass,classnyh,vdinsufm,vdinsuft,incidencREOP,status,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%,cvdcarsten,cvdstenrt,cvdstenlft,predstro,strokeBin2,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn,aortoccl,asmtaodx,cathbasassistind,cathbasassistwhen,cpbutil,cperftyp,ecmoind,ecmowhen,emergrsn,iabpind,iabpwhen,unplproc,urgntrsn
0,65,175.30000,79.4,25.83787,45.0,1.2,NaN,NaN,NaN,55.0,40.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,7,5,1,1,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,3,4,0,3,2,0,2,0,0,0,0,0,0,0,0,0,0,0.017,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,29.0,32.0,70.0,1.1,NaN,NaN,40.0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,

In [266]:
data.tail()

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,surgdt_month,surgdt_DayOfWeek,surgdt_PartOfMonth,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,numdisv,anginalclass,classnyh,vdinsufm,vdinsuft,incidencREOP,status,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%,cvdcarsten,cvdstenrt,cvdstenlft,predstro,strokeBin2,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn,aortoccl,asmtaodx,cathbasassistind,cathbasassistwhen,cpbutil,cperftyp,ecmoind,ecmowhen,emergrsn,iabpind,iabpwhen,unplproc,urgntrsn
28940,69,167.60001,84.5,30.08213,35.6,0.90,3.4,11.6,6.40,38.0,NaN,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,12,0,2,1,0,0,0,0,0,0,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,3,4,0,2,2,0,1,0,0,0,0,0,0,0,0,0,0,0.013,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,29.0,36.1,NaN,0.8,NaN,NaN,NaN,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0

In [267]:
data.shape

(28945, 249)

### Feature Matrix `X`
- in the next cell you can select which outcome vector you want to use: `strokeBin` or `strokeBin2`

In [268]:
X = data.copy().drop('strokeBin2', axis=1)

In [269]:
X = X.reset_index(drop=True)

In [270]:
X.head(2)

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,surgdt_month,surgdt_DayOfWeek,surgdt_PartOfMonth,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,numdisv,anginalclass,classnyh,vdinsufm,vdinsuft,incidencREOP,status,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%,cvdcarsten,cvdstenrt,cvdstenlft,predstro,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn,aortoccl,asmtaodx,cathbasassistind,cathbasassistwhen,cpbutil,cperftyp,ecmoind,ecmowhen,emergrsn,iabpind,iabpwhen,unplproc,urgntrsn
0,65,175.30000,79.4,25.83787,45.0,1.2,NaN,NaN,NaN,55.0,40.0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,1,0,7,5,1,1,1,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,3,4,0,3,2,0,2,0,0,0,0,0,0,0,0,0,0,0.017,NaN,0,0,NaN,NaN,NaN,NaN,NaN,29.0,32.0,70.0,1.1,NaN,NaN,40.0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0

In [271]:
X.shape

(28945, 248)

### Target Vector `y`

In [272]:
y = data.copy()['strokeBin2']

In [273]:
type(y)

pandas.core.series.Series

In [274]:
y.head(2)

0    0
1    0
Name: strokeBin2, dtype: int64

In [275]:
y.shape

(28945,)

In [276]:
y.unique()

array([0, 1])

In [277]:
y.value_counts()

0    28523
1      422
Name: strokeBin2, dtype: int64

### `X_train_all`, `X_devtest`, `y_train`, `y_devtest`
- using `train_test_split` with `stratify` parameter to ensure relative proportion of outcome classes are the same in `train`, `dev` and `test` sets
- observation split will be `80/10/10` between `train`, `dev`, `test`
- designation of `_all` denotes complete feature set (`PRE` + `POST`)

In [278]:
X_train_all, X_test_all, y_train, y_test = train_test_split(X,
                                                                  y,
                                                                  test_size=0.2,
                                                                  random_state=0,
                                                                  stratify=y)

#### validating `train_test_split`

In [279]:
X.shape

(28945, 248)

In [280]:
np.rint(X.shape[0] * 0.20)

5789.0

In [281]:
X_train_all.shape, X_test_all.shape, y_train.shape, y_test.shape

((23156, 248), (5789, 248), (23156,), (5789,))

In [282]:
X.shape[0] - X_train_all.shape[0] - X_test_all.shape[0]

0

In [283]:
y.shape[0] - y_train.shape[0] - y_test.shape[0]

0

#### validating `stratify` worked
- relative proportion of classes in `y`

In [284]:
print (np.round(y.value_counts()[0] / y.shape[0], 4))
print (np.round(y.value_counts()[1] / y.shape[0], 4))

0.9854
0.0146


- relative proportion of classes in `y_train`

In [285]:
print (np.round(y_train.value_counts()[0] / y_train.shape[0], 4))
print (np.round(y_train.value_counts()[1] / y_train.shape[0], 4))

0.9854
0.0146


- relative proportion of classes in `y_devtest`

In [286]:
print (np.round(y_test.value_counts()[0] / y_test.shape[0], 4))
print (np.round(y_test.value_counts()[1] / y_test.shape[0], 4))

0.9855
0.0145


### `X_dev_all`, `X_test_all`, `y_dev`, `y_test`

In [287]:
# X_dev_all, X_test_all, y_dev, y_test = train_test_split(X_devtest_all,
#                                                         y_devtest,
#                                                         test_size=0.5,
#                                                         random_state=0,
#                                                         stratify=y_devtest)

- validating `train_test_split`

In [288]:
# X_devtest_all.shape

In [289]:
# np.rint(X_devtest_all.shape[0] * 0.50)

In [290]:
# X_dev_all.shape, X_test_all.shape, y_dev.shape, y_dev.shape

#### validating `stratify` worked
- relative proportion of classes in `y`

In [291]:
# print (np.round(y.value_counts()[0] / y.shape[0], 4))
# print (np.round(y.value_counts()[1] / y.shape[0], 4))

- relative proportion of classes in `y_devtest`

In [292]:
# print (np.round(y_devtest.value_counts()[0] / y_devtest.shape[0], 4))
# print (np.round(y_devtest.value_counts()[1] / y_devtest.shape[0], 4))

- relative proportion of classes in `y_dev`

In [293]:
# print (np.round(y_dev.value_counts()[0] / y_dev.shape[0], 4))
# print (np.round(y_dev.value_counts()[1] / y_dev.shape[0], 4))

- relative proportion of classes in `y_test`

In [294]:
# print (np.round(y_test.value_counts()[0] / y_test.shape[0], 4))
# print (np.round(y_test.value_counts()[1] / y_test.shape[0], 4))

### Resetting Indicies

In [295]:
X_train_all = X_train_all.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)

In [296]:
# X_dev_all = X_dev_all.reset_index(drop=True)
# y_dev = y_dev.reset_index(drop=True)

In [297]:
X_test_all = X_test_all.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

### Extracting `predstro` from `X_train`, `X_dev` and `X_test`
- and subsequently creating `predstro_train`, `predstro_dev`, `predstro_test`

- `train`

In [298]:
X_train_all.shape

(23156, 248)

In [299]:
predstro_train = X_train_all['predstro']

In [300]:
predstro_train.shape

(23156,)

In [301]:
X_train_all = X_train_all.drop('predstro', axis=1)

In [302]:
X_train_all.head(2)

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,surgdt_month,surgdt_DayOfWeek,surgdt_PartOfMonth,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,numdisv,anginalclass,classnyh,vdinsufm,vdinsuft,incidencREOP,status,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%,cvdcarsten,cvdstenrt,cvdstenlft,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn,aortoccl,asmtaodx,cathbasassistind,cathbasassistwhen,cpbutil,cperftyp,ecmoind,ecmowhen,emergrsn,iabpind,iabpwhen,unplproc,urgntrsn
0,57,177.80000,130.10001,41.15417,31.0,1.00,3.9,7.3,7.50,50.0,42.0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,12,0,1,1,1,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,3,4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,NaN,0,0,NaN,0.0,3.0,0.0,0.0,22.0,34.0,106.0,1.2,73.0,75.0,93.0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1

In [303]:
X_train_all.shape

(23156, 247)

- `dev`

In [304]:
#X_dev_all.shape

In [305]:
#predstro_dev = X_dev_all['predstro']

In [306]:
#predstro_dev.shape

In [307]:
#X_dev_all = X_dev_all.drop('predstro', axis=1)

In [308]:
#X_dev_all.head(2)

In [309]:
#X_dev_all.shape

- `test`

In [310]:
X_test_all.shape

(5789, 248)

In [311]:
predstro_test = X_test_all['predstro']

In [312]:
predstro_test.shape

(5789,)

In [313]:
X_test_all = X_test_all.drop('predstro', axis=1)

In [314]:
X_test_all.head(2)

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,surgdt_month,surgdt_DayOfWeek,surgdt_PartOfMonth,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,numdisv,anginalclass,classnyh,vdinsufm,vdinsuft,incidencREOP,status,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%,cvdcarsten,cvdstenrt,cvdstenlft,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn,aortoccl,asmtaodx,cathbasassistind,cathbasassistwhen,cpbutil,cperftyp,ecmoind,ecmowhen,emergrsn,iabpind,iabpwhen,unplproc,urgntrsn
0,62,177.8,91.1,28.81740,40.6,1.0,3.6,NaN,9.34,58.0,NaN,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,3,6,1,1,1,0,0,0,0,0,0,1,0,1,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,2,3,0,2,2,0,1,0,0,0,0,0,0,0,0,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,29.0,36.0,48.0,1.1,NaN,NaN,34.0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,2,0,0,0,

In [315]:
X_test_all.shape

(5789, 247)

- validating row count - `42,740` total observations

In [316]:
42740 - X_train_all.shape[0]  - X_test_all.shape[0]#- X_dev_all.shape[0]

13795

#### Last look at the data (`X_train_all`) before modeling

In [317]:
X_train_all.head()

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,surgdt_month,surgdt_DayOfWeek,surgdt_PartOfMonth,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,numdisv,anginalclass,classnyh,vdinsufm,vdinsuft,incidencREOP,status,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%,cvdcarsten,cvdstenrt,cvdstenlft,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn,aortoccl,asmtaodx,cathbasassistind,cathbasassistwhen,cpbutil,cperftyp,ecmoind,ecmowhen,emergrsn,iabpind,iabpwhen,unplproc,urgntrsn
0,57,177.80000,130.10001,41.15417,31.0,1.00,3.9,7.3,7.50,50.0,42.0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,12,0,1,1,1,0,0,0,0,0,1,1,0,1,0,1,0,0,0,0,1,0,0,1,1,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,3,4,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,NaN,0,0,NaN,0.0,3.0,0.0,0.0,22.0,34.0,106.0,1.2,73.0,75.0,93.0,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1

In [318]:
X_train_all.tail()

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,surgdt_month,surgdt_DayOfWeek,surgdt_PartOfMonth,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,numdisv,anginalclass,classnyh,vdinsufm,vdinsuft,incidencREOP,status,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%,cvdcarsten,cvdstenrt,cvdstenlft,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn,aortoccl,asmtaodx,cathbasassistind,cathbasassistwhen,cpbutil,cperftyp,ecmoind,ecmowhen,emergrsn,iabpind,iabpwhen,unplproc,urgntrsn
23151,66,175.2,134.7,43.88331,34.2,0.80,4.0,6.9,7.47,62.5,27.1,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,1,0,8,1,1,1,1,0,0,0,0,0,1,1,0,1,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,3,3,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,NaN,0,0,NaN,NaN,NaN,NaN,NaN,24.0,34.1,99.0,0.8,NaN,NaN,56.0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,1,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,2

## Pre-processing Data using `Scikit-Learn` `Pipelines`

### Creating a `FeatureSelector` transformer
- necessary because we are working with heterogeneous data (numerical and categorical features) -- want to be able to pick and choose which features (columns) to pass through our pipelines (and transform them) instead of having to pass through the whole dataframe

In [319]:
class FeatureSelector(BaseEstimator, TransformerMixin):
    '''
    Transformer to select column from a data frame to perform additional transformations on
    Use this for selecting column(s) that require fit transform
    '''
    
    def __init__(self, key):
        self.key = key

    def fit(self, x, y=None):
        return self

    def transform(self, data_dict):
        return data_dict[data_dict.columns.intersection(self.key)]

## Loading Column Names for `PREOP` and `POSTOP` Features
- these `lists` of features will be used to mask/select specified features from the `COMBO` dataset to assemble `PREOP`, `PREOP_TREE`, `POSTOP` and `POSTOP_TREE` feature sets as appropriate
- these are essentially helper lists as `DataFrames` put through `Pipelines` emerge as `numpy` arrays which do not have column names
- can convert resulting `numpy` arrays back into `DataFrames` for further analysis

#### `PREOP_dataset` Column Names

- `DUMMIES` Version = `PREOP_numerical_col_names` + `PREOP_categorical_col_names`
- `TREE` Version    = `PREOP_numerical_col_names` + `PREOP_categorical_TREE_col_names`

- `PREOP_numerical_col_names`

In [320]:
with open('PREOP_numerical_col_names_12_1.pkl', 'rb') as filehandle:
    PREOP_numerical_col_names = pickle.load(filehandle)

In [321]:
PREOP_numerical_col_names[0:5]

['age', 'heightcm', 'weightkg', 'bmi', 'hct']

In [322]:
len(PREOP_numerical_col_names)

11

- `PREOP_categorical_col_names`

In [323]:
with open('PREOP_categorical_col_names_12_1.pkl', 'rb') as filehandle:
    PREOP_categorical_col_names = pickle.load(filehandle)

In [324]:
PREOP_categorical_col_names[0:5]

['surgdt_month_Jan',
 'surgdt_month_Feb',
 'surgdt_month_Mar',
 'surgdt_month_Apr',
 'surgdt_month_May']

In [325]:
len(PREOP_categorical_col_names)

99

- `PREOP_categorical_TREE_col_names`

In [326]:
with open('PREOP_categorical_TREE_col_names_12_1.pkl', 'rb') as filehandle:
    PREOP_categorical_TREE_col_names = pickle.load(filehandle)

In [327]:
PREOP_categorical_TREE_col_names[0:5]

['surgdt_month',
 'surgdt_DayOfWeek',
 'surgdt_PartOfMonth',
 'gender',
 'racecaucasian']

In [328]:
len(PREOP_categorical_TREE_col_names)

61

#### `POSTOP_dataset` Column Names

- `DUMMIES` Version = `POSTOP_numerical_col_names` + `POSTOP_categorical_col_names`
- `TREE` Version = `POSTOP_numerical_col_names` + `POSTOP_categorical_TREE_col_names`

In [329]:
with open('POSTOP_numerical_col_names_12_1.pkl', 'rb') as filehandle:
    POSTOP_numerical_col_names = pickle.load(filehandle)

In [330]:
with open('POSTOP_categorical_col_names_12_1.pkl', 'rb') as filehandle:
    POSTOP_categorical_col_names = pickle.load(filehandle)

In [331]:
with open('POSTOP_categorical_TREE_col_names_12_1.pkl', 'rb') as filehandle:
    POSTOP_categorical_TREE_col_names = pickle.load(filehandle)

## Functions Used in Pipelines

#### `SimpleImputer(missing_values=np.nan, strategy='median'` 
- to replace numerical feature `NaN`s with `X_train` numerical feature `NaN`s

#### `PolynomialFeatures()`
- per Albon 13.2 pages 225-227, can use `PolynomialFeatures()` to create interaction terms
- Parameter `interaction_only=True` tells `PolynomialFeatures()` to ONLY return interaction terms (and not polynomial features
- By default, `PolynomialFeatures()` will add a feature containing ones (a vector of `1`s) called a `bias` -- we can prevent that through the parameter `include_bias=False`
- The `degree` parameter determines the maximum number of features to create interaction terms from (in case we wanted to create an interaction term of `n_features`)
- open question whether we should apply `PolynomialFeatures` to the entire feature matrix or only to the numerical and/or categorical features and how we should do it (with everything higher degree + interaction terms or just interaction terms).
- Given the number of featues we have, we will have many more features after transforming with `PolynomialFeatures`
- The formula for calculating the number of the polynomial features is `N(n,d)=C(n+d,d)` where `n` is the number of the features, `d` is the degree of the polynomial, `C` is binomial coefficient(combination). In our case the number is `C(3+2,2)=5!/(5-2)!2!=10` but when the number of features or the degree is high the polynomial features becomes too many.

#### `sklearn.preprocessing` scalers such as `StandardScaler()`

#### Custom `helper` functions, such as `convert_df_to_numpy`
- need to create a function to convert `DataFrame` to `numpy.ndarray` then can use `FeatureUnion` to combine with `numpy.ndarray` that results from `numerical_pipeline` to form feature matrix

In [332]:
def convert_df_to_numpy(df):
    
    df = df.values
    
    return df

#### `FeatureUnion`
- to reassemble feature matrix from the product of multiple `pipelines`

# `PREOP` Feature Matrix Pipelines

### `PREOP_numerical_features` Pipeline

In [333]:
PREOP_numerical_features_pipeline = Pipeline(steps=[
    ('select', FeatureSelector(PREOP_numerical_col_names)), 
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='median')),
    ('scaler', StandardScaler()) 
    ])

- `fit_transform` on `train` data, `transform` only on `dev` and `test` data

In [334]:
PREOP_numerical = PREOP_numerical_features_pipeline.fit_transform(X_train_all)

In [335]:
PREOP_numerical.shape, len(PREOP_numerical_col_names)

((23156, 11), 11)

### `PREOP_categorical_features_pipeline`

In [336]:
PREOP_categorical_features_pipeline = Pipeline(steps=[
    ('select', FeatureSelector(PREOP_categorical_col_names)),
    ('convert_numpy', FunctionTransformer(convert_df_to_numpy, validate=False))
    ])

- `fit_transform` on `train` data, `transform` only on `dev` and `test` data

In [337]:
PREOP_categorical = PREOP_categorical_features_pipeline.fit_transform(X_train_all)

In [338]:
PREOP_categorical.shape, len(PREOP_categorical_col_names)

((23156, 99), 99)

## Assembling `PREOP_feature_matrix` via `FeatureUnion`

In [339]:
PREOP_feature_matrix = FeatureUnion([('PREOP_num_features', PREOP_numerical_features_pipeline),
                                     ('PREOP_cat_features', PREOP_categorical_features_pipeline)
                                    ])

In [340]:
type(PREOP_feature_matrix)

sklearn.pipeline.FeatureUnion

### Transforming `X_train_all` to get `X_train_PREOP`

In [341]:
X_train_PREOP = PREOP_feature_matrix.fit_transform(X_train_all)

- validating

In [342]:
X_train_PREOP.shape, y_train.shape

((23156, 110), (23156,))

- putting into a `DataFrame` for easy analysis

In [343]:
PREOP_feature_col_names = PREOP_numerical_col_names + PREOP_categorical_col_names

In [344]:
len(PREOP_feature_col_names), len(PREOP_numerical_col_names), len(PREOP_categorical_col_names)

(110, 11, 99)

In [345]:
X_train_PREOP = pd.DataFrame(X_train_PREOP,
                             columns=PREOP_feature_col_names)

In [346]:
X_train_PREOP.head()

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%
0,-0.82818,0.55124,1.96124,0.97889,-1.47015,-0.14268,0.31205,0.52945,-0.27512,-0.16511,0.97676,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.80602,-1.38063,-2.30378,-1.22765,-0.27614,-0.46271,-0.47966,-0.41307,-0.60388,-0.00060,-0.07777,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.02552,1.99068,-0.27282,-0.75045,0.58206,0.18838,-0.87552,-0.27843,1.03243,-0.98767,-0.07777,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.73283,0.55124,0.81697,0.28968,0.58206,0.40909,0.70791,-0.68237,1.16692,0.90420,-0.91762,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-1.31599,1.80128,0.56929,-0.26561,0.39550,-0.58409,-1.46931,3.69365,0.06111,-2.63277,-0.07777,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Transforming `X_dev_all` to get `X_dev_PREOP`

In [347]:
#X_dev_PREOP = PREOP_feature_matrix.transform(X_dev_all)

- validating

In [348]:
#X_dev_PREOP.shape, y_dev.shape

- putting into a `DataFrame` for easy analysis

In [349]:
# X_dev_PREOP = pd.DataFrame(X_dev_PREOP,
#                            columns=PREOP_feature_col_names)

In [350]:
# X_dev_PREOP.head()

### Transforming `X_test_all` to get `X_test_PREOP`

In [351]:
X_test_PREOP = PREOP_feature_matrix.transform(X_test_all)

- validating

In [352]:
X_test_PREOP.shape, y_test.shape

((5789, 110), (5789,))

- putting into a `DataFrame` for easy analysis

In [353]:
X_test_PREOP = pd.DataFrame(X_test_PREOP,
                            columns=PREOP_feature_col_names)

In [354]:
X_test_PREOP.head()

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%
0,-0.34036,0.55124,0.02935,-0.18472,0.32087,-0.14268,-0.28173,-0.34575,0.41228,0.49293,-0.07777,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.02552,-0.16847,1.35691,0.93606,-0.35076,0.62979,0.70791,0.79875,1.74225,-0.98767,0.85047,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.53549,-0.85031,-0.57003,-0.09827,-0.33211,-0.49581,-1.66724,-0.81701,-0.56278,1.72676,-0.91762,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-1.70625,1.04368,-0.29759,-0.52288,-0.35076,-0.36339,-0.47966,-0.34575,-0.68607,-3.24969,-0.15986,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.04989,0.57018,0.20272,-0.08663,0.76863,-0.47374,0.50998,-0.34575,-0.68607,1.23322,-1.54908,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


#### NOTE:
- did not have to put feature matricies into `DataFrames` - could have used directly in pipelines, but wanted to put into `DataFrames` in case wanted to retrieve feature names

# `POSTOP` Feature Matrix Pipelines

### `POSTOP_numerical_features` Pipeline

In [355]:
POSTOP_numerical_features_pipeline = Pipeline(steps=[
    ('select', FeatureSelector(POSTOP_numerical_col_names)), 
    ('imputer', SimpleImputer(missing_values=np.nan, strategy='median')),
    ('scaler', StandardScaler()) 
    ])

- `fit_transform` on `train` data, `transform` only on `dev` and `test` data

In [356]:
POSTOP_numerical = POSTOP_numerical_features_pipeline.fit_transform(X_train_all)

In [357]:
POSTOP_numerical.shape, len(POSTOP_numerical_col_names)

((23156, 15), 15)

### `POSTOP_categorical_features_pipeline`

In [358]:
POSTOP_categorical_features_pipeline = Pipeline(steps=[
    ('select', FeatureSelector(POSTOP_categorical_col_names)),
    ('convert_numpy', FunctionTransformer(convert_df_to_numpy, validate=False))
    ])

- `fit_transform` on `train` data, `transform` only on `dev` and `test` data

In [359]:
POSTOP_categorical = POSTOP_categorical_features_pipeline.fit_transform(X_train_all)

In [360]:
POSTOP_categorical.shape, len(POSTOP_categorical_col_names)

((23156, 96), 96)

## Assembling `POSTOP_feature_matrix` via `FeatureUnion`

In [361]:
POSTOP_feature_matrix = FeatureUnion([('POSTOP_num_features', POSTOP_numerical_features_pipeline),
                                      ('POSTOP_cat_features', POSTOP_categorical_features_pipeline)
                                     ])

In [362]:
type(POSTOP_feature_matrix)

sklearn.pipeline.FeatureUnion

### Transforming `X_train_all` to get `X_train_POSTOP`

In [363]:
X_train_POSTOP = POSTOP_feature_matrix.fit_transform(X_train_all)

- validating

In [364]:
X_train_POSTOP.shape, y_train.shape

((23156, 111), (23156,))

- putting into a `DataFrame` for easy analysis

In [365]:
POSTOP_feature_col_names = POSTOP_numerical_col_names + POSTOP_categorical_col_names

In [366]:
len(POSTOP_feature_col_names), len(POSTOP_numerical_col_names), len(POSTOP_categorical_col_names)

(111, 15, 96)

In [367]:
X_train_POSTOP = pd.DataFrame(X_train_POSTOP,
                              columns=POSTOP_feature_col_names)

In [368]:
X_train_POSTOP.head()

,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn
0,-0.00693,-0.0698,-0.06628,-0.00465,-0.03571,5.70929,-0.19482,-2.84486,-0.76826,-0.00312,0.11526,-0.16015,2.00945,2.55393,0.53611,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,-0.00693,-0.0698,-0.06628,-0.00465,-0.03571,-0.18655,-0.19482,0.12646,-0.36199,0.26499,-0.80508,-0.52011,0.04493,0.03835,-0.42841,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.00693,-0.0698,-0.06628,-0.00465,-0.03571,-0.18655,11.74946,-2.84486,0.04429,-0.67341,4.56355,0.91973,0.04493,0.03835,4.23865,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
3,-0.00693,-0.0698,-0.06628,-0.00465,-0.03571,-0.18655,4.58289,-2.84486,-0.76826,0.73420,-0.62612,0.55977,0.04493,0.03835,-0.77067,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.00693,-0.0698,-0.06628,-0.00465,-0.03571,-0.18655,-0.19482,0.12646,1.26310,-0.00312,0.19195,-0.61010,0.04493,0.03835,-0.55287,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,

### Transforming `X_dev_all` to get `X_dev_POSTOP`

In [369]:
# X_dev_POSTOP = POSTOP_feature_matrix.transform(X_dev_all)

- validating

In [370]:
# X_dev_POSTOP.shape, y_dev.shape

- putting into a `DataFrame` for easy analysis

In [371]:
# X_dev_POSTOP = pd.DataFrame(X_dev_POSTOP,
#                             columns=POSTOP_feature_col_names)

In [372]:
# X_dev_POSTOP.head()

### Transforming `X_test_all` to get `X_test_POSTOP`

In [373]:
X_test_POSTOP = POSTOP_feature_matrix.transform(X_test_all)

- validating

In [374]:
X_test_POSTOP.shape, y_test.shape

((5789, 111), (5789,))

- putting into a `DataFrame` for easy analysis

In [375]:
X_test_POSTOP = pd.DataFrame(X_test_POSTOP,
                             columns=POSTOP_feature_col_names)

In [376]:
X_test_POSTOP.head()

,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn
0,-0.00693,-0.0698,-0.06628,-0.00465,-0.03571,-0.18655,-0.19482,0.12646,0.65369,1.33746,-1.36751,-0.25014,0.04493,0.03835,-1.29960,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.00693,-0.0698,-0.06628,-0.00465,-0.03571,-0.18655,-0.19482,0.12646,-0.56512,-0.00312,2.33940,0.46978,0.04493,0.03835,2.15403,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.00693,-0.0698,-0.06628,-0.00465,-0.03571,-0.18655,0.99960,0.12646,-1.78393,0.33202,-0.16596,0.55977,0.04493,0.03835,-0.86401,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.00693,-0.0698,-0.06628,-0.00465,-0.03571,-0.18655,-0.19482,0.12646,-0.76826,-1.07559,1.01003,-0.43012,0.04493,0.03835,0.50500,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.00693,-0.0698,-0.06628,-0.00465,-0.03571,-0.18655,-0.19482,0.12646,0.04429,0.46608,-0.42161,-0.52011,0.04493,0.03835,-0.21062,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.

## Assembling `PREOP + POSTOP` Combined Feature Matrix

#### -  Using `FeatureUnion | Pipeline` to Create Combined `PREOP + POSTOP` Feature Matrix

In [377]:
PREPOST_feature_matrix = FeatureUnion([('PREOP_num_features', PREOP_numerical_features_pipeline),
                                       ('PREOP_cat_features', PREOP_categorical_features_pipeline),
                                       ('POSTOP_num_features', POSTOP_numerical_features_pipeline),
                                       ('POSTOP_cat_features', POSTOP_categorical_features_pipeline)
                                      ])

- can use the code pattern above to create `X_train_PREPOST`, `X_dev_PREPOST`, `X_test_PREPOST`

In [378]:
PREPOST_feature_col_names = PREOP_feature_col_names + POSTOP_feature_col_names

In [379]:
len(PREPOST_feature_col_names), len(PREOP_feature_col_names), len(POSTOP_feature_col_names)

(221, 110, 111)

### Transforming `X_train_all` to get `X_train_PREPOST`

In [380]:
X_train_PREPOST = PREPOST_feature_matrix.fit_transform(X_train_all)

- validating

In [381]:
X_train_PREPOST.shape, y_train.shape

((23156, 221), (23156,))

- putting into a `DataFrame` for easy analysis

In [382]:
X_train_PREPOST = pd.DataFrame(X_train_PREPOST,
                               columns=PREPOST_feature_col_names)

In [383]:
X_train_PREPOST.head()

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn
0,-0.82818,0.55124,1.96124,0.97889,-1.47015,-0.14268,0.31205,0.52945,-0.27512,-0.16511,0.97676,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.00693,-0.0698,-0.06628,-0.00465,-0.03571,5.70929,-0.19482,-2.84486,-0.76826,-0.00312,0.11526,-0.16015,2.00945,2.55393,0.53611,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,

### Transforming `X_dev_all` to get `X_dev_PREPOST`

In [384]:
# X_dev_PREPOST = PREPOST_feature_matrix.transform(X_dev_all)

- validating

In [385]:
# X_dev_PREPOST.shape, y_dev.shape

- putting into a `DataFrame` for easy analysis

In [386]:
# X_dev_PREPOST = pd.DataFrame(X_dev_PREPOST,
#                              columns=PREPOST_feature_col_names)

In [387]:
# X_dev_PREPOST.head()

### Transforming `X_test_all` to get `X_test_PREPOST`

In [388]:
X_test_PREPOST = PREPOST_feature_matrix.transform(X_test_all)

- validating

In [389]:
X_test_PREPOST.shape, y_test.shape

((5789, 221), (5789,))

- putting into a `DataFrame` for easy analysis

In [390]:
X_test_PREPOST = pd.DataFrame(X_test_PREPOST,
                              columns=PREPOST_feature_col_names)

In [391]:
X_test_PREPOST.head()

,age,heightcm,weightkg,bmi,hct,creatlst,totalbumin,a1clvl,meldscr,hdef,pasys,surgdt_month_Jan,surgdt_month_Feb,surgdt_month_Mar,surgdt_month_Apr,surgdt_month_May,surgdt_month_Jul,surgdt_month_Aug,surgdt_month_Sep,surgdt_month_Oct,surgdt_month_Nov,surgdt_month_Dec,surgdt_DayOfWeek_Mon,surgdt_DayOfWeek_Tues,surgdt_DayOfWeek_Thurs,surgdt_DayOfWeek_Fri,surgdt_DayOfWeek_Sat,surgdt_DayOfWeek_Sun,surgdt_PartOfMonth_Beg,surgdt_PartOfMonth_End,gender,racecaucasian,raceblack,raceasian,racenativeam,racnativepacific,ethnicity,diabetes,dyslip,dialysis,hypertn,infendo,slpapn,liverdis,immsupp,mediastrad,cancer,pvd,syncope,unrespstat,cvd,cva,cvdtia,cvdpcarsurg,hitanti,prcvint,prcab,prvalve,chf,priorhf,arrhyafib,medinotr,hdefd,vdaort,vdstena,vdstenm,diabctrl,infendty,Tobacco_Combined,chrlungd,hmo2,ivdrugab,alcohol,carshock24,resusc24,medasa,medaplt5days,medlipid,numdisv_1_CORONARY,numdisv_2_CORONARIES,numdisv_3_CORONARIES,anginalclass_STRENUOUS_ACTIVITY,anginalclass_SLIGHT_LIMITATION_ACTIVITY,anginalclass_MARKED_LIMITATION_ACTIVITY,anginalclass_ANGINA_AT_REST,classnyh_SLIGHT_LIMITATION,classnyh_MARKED_LIMITATION,classnyh_ANY_ACTIVITY,vdinsufm_TRIVIAL,vdinsufm_MILD,vdinsufm_MODERATE,vdinsufm_SEVERE,vdinsuft_TRIVIAL,vdinsuft_MILD,vdinsuft_MODERATE,vdinsuft_SEVERE,incidencREOP_FIRST,incidencREOP_SECOND,incidencREOP_THIRD,incidencREOP_FOURTH,status_URGENT,status_EMERGENCY,status_SALVAGE,cvdcarsten_RIGHT,cvdcarsten_LEFT,cvdcarsten_BOTH,cvdstenrt_80-99%,cvdstenrt_100%,cvdstenlft_80-99%,cvdstenlft_100%,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn
0,-0.34036,0.55124,0.02935,-0.18472,0.32087,-0.14268,-0.28173,-0.34575,0.41228,0.49293,-0.07777,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.00693,-0.0698,-0.06628,-0.00465,-0.03571,-0.18655,-0.19482,0.12646,0.65369,1.33746,-1.36751,-0.25014,0.04493,0.03835,-1.29960,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0

## Modeling Scenarios Using `PREOP` and `POSTOP` Datasets

### - Scenario A: `Kitchen Sink` - Modeling the Combined `PREOP` and `POSTOP` Dataset

- code belows shows feature matrix assembly using `Pipeline` - could have used the already named `feature matricies` created above: `X_train_PREPOST`, `X_dev_PREPOST` and `X_test_PREPOST` in conjunction with their respective `y` vectors

#### `LogisticRegression()` with `GridSearchCV`

- instantiating `Pipeline`

In [392]:
PREPOST_modelA_pipe = Pipeline(steps=[
    ('get_PREPOST_feature_matrix', PREPOST_feature_matrix),
    ('Classifier', LogisticRegression())
    ])

- defining a parameter grid for `GridSearchCV`

In [393]:
parameter_grid = {'Classifier__penalty': ['l1'],
                  'Classifier__C': [0.001, 0.01, 1.0],
                  'Classifier__class_weight': ['balanced'],
                  'Classifier__solver': ['liblinear'],
                  'Classifier__random_state': [0]
                 }

- defining `scoring_metrics`

In [394]:
scoring_metrics = ['accuracy',
                   'f1',
                   'f1_macro',
                   'f1_weighted',
                   'precision',
                   'precision_macro',
                   'precision_weighted',
                   'recall',
                   'recall_macro',
                   'recall_weighted',
                   'roc_auc']

- create `GridSearchCV` object

In [395]:
gs = GridSearchCV(PREPOST_modelA_pipe, #pipline
                  param_grid=parameter_grid, #param_grid
                  scoring=scoring_metrics, #loss functions
                  cv=5, #number of folds, default=`StratifiedKFold`
                  refit='roc_auc', # best_estimator_ will be based on this scoring metric
                  return_train_score=True,
                  n_jobs=-1,
                  verbose=5)

- fitting `GridSearchCV`

In [396]:
PREPOST_modelA = gs.fit(X_train_all, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  15 | elapsed:  1.6min remaining:   23.8s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:  2.1min finished


- examining results

In [397]:
gs_results = pd.DataFrame(PREPOST_modelA.cv_results_).sort_values(by=['mean_test_roc_auc'],
                                                                  ascending=False)

In [398]:
display_cols = ['param_Classifier__C',
                'param_Classifier__penalty',
                'mean_test_roc_auc',
                'mean_train_roc_auc',
                'mean_test_f1',
                'mean_test_precision',
                'mean_test_recall']

In [399]:
gs_results_summary = gs_results.copy()[display_cols].reset_index(drop=True)

In [400]:
gs_results_summary.rename(columns={'param_Classifier__C': 'C',
                                   'param_Classifier__penalty': 'penalty'},
                          inplace=True)

In [401]:
gs_results_summary

,C,penalty,mean_test_roc_auc,mean_train_roc_auc,mean_test_f1,mean_test_precision,mean_test_recall
0,0.01,l1,0.72034,0.78303,0.06019,0.03177,0.57094
1,1,l1,0.67295,0.84717,0.05789,0.03072,0.50298
2,0.001,l1,0.63518,0.65901,0.03918,0.02019,0.65983


#### Key Takeaways
- `LogisticRegression()` with `C=0.01` beats `STS`

## Scenario Ab - Just Preop Data Logistic

### Step 3b: Model and Evaluate Stacked Model Using `RandomForestClassifier`

In [402]:
PRE_feature_matrix = FeatureUnion([('PREOP_num_features', PREOP_numerical_features_pipeline),
                                       ('PREOP_cat_features', PREOP_categorical_features_pipeline)
                                      ])

In [403]:
PRE_modelA_pipe = Pipeline(steps=[
    ('get_PRE_feature_matrix', PRE_feature_matrix),
    ('Classifier', LogisticRegression())
    ])

In [404]:
parameter_grid = {'Classifier__penalty': ['l1'],
                  'Classifier__C': [0.0275, 0.01, 1.0],
                  'Classifier__class_weight': ['balanced'],
                  'Classifier__solver': ['liblinear'],
                  'Classifier__random_state': [0]
                 }

In [405]:
scoring_metrics = ['accuracy',
                   'f1',
                   'f1_macro',
                   'f1_weighted',
                   'precision',
                   'precision_macro',
                   'precision_weighted',
                   'recall',
                   'recall_macro',
                   'recall_weighted',
                   'roc_auc']

In [406]:
gs = GridSearchCV(PRE_modelA_pipe, #pipline
                  param_grid=parameter_grid, #param_grid
                  scoring=scoring_metrics, #loss functions
                  cv=5, #number of folds, default=`StratifiedKFold`
                  refit='roc_auc', # best_estimator_ will be based on this scoring metric
                  return_train_score=True,
                  n_jobs=-1,
                  verbose=5)

In [407]:
PRE_modelA = gs.fit(X_train_all, y_train)

Fitting 5 folds for each of 3 candidates, totalling 15 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  12 out of  15 | elapsed:   23.3s remaining:    5.8s
[Parallel(n_jobs=-1)]: Done  15 out of  15 | elapsed:   26.9s finished


In [408]:
gs_results = pd.DataFrame(PRE_modelA.cv_results_).sort_values(by=['mean_test_roc_auc'],
                                                                  ascending=False)
display_cols = ['param_Classifier__C',
                'param_Classifier__penalty',
                'mean_test_roc_auc',
                'mean_train_roc_auc',
                'mean_test_f1',
                'mean_test_precision',
                'mean_test_recall']
gs_results_summary = gs_results.copy()[display_cols].reset_index(drop=True)
gs_results_summary.rename(columns={'param_Classifier__C': 'C',
                                   'param_Classifier__penalty': 'penalty'},
                          inplace=True)

In [409]:
gs_results_summary

,C,penalty,mean_test_roc_auc,mean_train_roc_auc,mean_test_f1,mean_test_precision,mean_test_recall
0,0.0275,l1,0.69219,0.76608,0.05075,0.02657,0.56199
1,0.01,l1,0.68186,0.73244,0.04799,0.02505,0.57103
2,1,l1,0.67653,0.78839,0.05123,0.02688,0.54400


### - Scenario B: `Model Stacking` - Incorporate Predictions from `PREOP` Training with `POSTOP` Data
- this time, we will make use of pre-saved `X_train_PREOP` and `X_train_POSTOP` feature matricies created above via `Pipeline` 

In [410]:
X_train_PREOP.shape, X_train_POSTOP.shape, y_train.shape

((23156, 110), (23156, 111), (23156,))

#### Step 1.  Generate vector of probabilities of stroke for using `PREOP` features and cross validated training set
- `LogisticRegression` hyperparameters were tuned in a separate notebook

In [411]:
lr_predict_probs = cross_val_predict(LogisticRegression(penalty='l1',
                                                        C=0.0275,
                                                        class_weight='balanced',
                                                        random_state=0,
                                                        solver='liblinear'),
                                     X_train_PREOP,
                                     y_train,
                                     cv=5,
                                     method='predict_proba',
                                     n_jobs=-1,
                                     verbose=5)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:    1.1s remaining:    1.7s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:    3.3s finished


- `cross_val_predict` returns an `numpy.ndarray` with `shape` `(n, 2)` as it returns a probability per class `(0, 1)`
- since we are interested in the `1` class, need to carve our the second column of the `numpy.ndarray`
- add `[:,1]`

- examining `lr_predict_probs`

In [412]:
type(lr_predict_probs)

numpy.ndarray

In [413]:
lr_predict_probs.shape

(23156, 2)

In [414]:
lr_predict_probs[0:5]

array([[0.42844504, 0.57155496],
       [0.25829605, 0.74170395],
       [0.51187637, 0.48812363],
       [0.63773056, 0.36226944],
       [0.5024975 , 0.4975025 ]])

In [415]:
lr_predict_probs[0:5, 1]

array([0.57155496, 0.74170395, 0.48812363, 0.36226944, 0.4975025 ])

### Step 1a.  Generate additional vector of probabilities of stroke for using `PREOP` features and CV training set
- Not limited to stacking one `PREOP` model
- Running `RandomForestClassifier` `PREOP` model 
- hyperparameters were tuned in a separate notebook - using AP's highest scoring model
- going to assemble `X_train_PREOP_TREE` using `Pipelines`
- can utilize existing `PREOP_numerical_features_pipeline`
- need to define `PREOP_categorial_TREE_features_pipeline`
- use`FeatureUnion` to combine to create `PREOP` feature matrix usable for `RandomForestClassifier`

- creating `Pipeline` to assemble `PREOP_categorical_TREE_features_pipeline`

In [416]:
PREOP_categorical_TREE_features_pipeline = Pipeline(steps=[
    ('select', FeatureSelector(PREOP_categorical_TREE_col_names)),
    ('convert_numpy', FunctionTransformer(convert_df_to_numpy, validate=False))
    ])

In [417]:
PREOP_feature_matrix_TREE = FeatureUnion([('PREOP_num_features', PREOP_numerical_features_pipeline),
                                          ('PREOP_cat_tree_features', PREOP_categorical_TREE_features_pipeline)
                                         ])

- combining with `PREOP_numerical_features_pipeline` defined above to assemble `PREOP_feature_matrix_TREE`
- if plan is to assemble `feature_matrix` but not use in a modeling pipeline as is done below, then can assign to a variable name like `X_train_PREOP_TREE` -- did this above for `X_train_PREOP` 

#### Modeling `PREOP_dataset_TREE` using `RandomForestClassifier()`

- creating `PREOP_RF_CLF_pipe`

In [418]:
PREOP_RF_clf_pipe = Pipeline(steps=[
    ('get_PREOP_TREE_feature_matrix', PREOP_feature_matrix_TREE),
    ('Classifier', RandomForestClassifier(n_estimators=315,
                                          max_features=0.1,
                                          max_depth=None,
                                          min_samples_split=50,
                                          class_weight='balanced',
                                          random_state=0))])

- in this scenario - assuming already have tuned the hyperparamenters
- alternatively, could have run a `GridSearchCV` first to tune the hyperparameters and then use the `best_estimator_` to get estimated probabilities of stroke with the code below

- running `RandomForestClassifer`

In [419]:
rf_predict_probs = cross_val_predict(PREOP_RF_clf_pipe,
                                     X_train_all,
                                     y_train,
                                     cv=5,
                                     method='predict_proba',
                                     n_jobs=-1,
                                     verbose=5)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   14.0s remaining:   21.0s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   23.1s finished


- examining `rf_predict_probs`

In [420]:
rf_predict_probs.shape

(23156, 2)

In [421]:
rf_predict_probs[0:5, 1]

array([0.21333341, 0.29233191, 0.19102064, 0.0751489 , 0.17894819])

### Step 2: Incorporate vector of predicted probabilities into `X_train_POSTOP`
- adding `lr_predict_probs`
- adding `rf_predict_probs`
- adding `STS` model predicted probability using `predstro_train`

In [422]:
X_train_POSTOP_STACKED = X_train_POSTOP.copy()

In [423]:
X_train_POSTOP_STACKED.shape, X_train_POSTOP.shape

((23156, 111), (23156, 111))

- adding vectors of `predict_probas` from `PREOP` models and `STS` models to `POSTOP` feature matrix

In [424]:
X_train_POSTOP_STACKED['LR_PREOP_PROBA'] = lr_predict_probs[:, 1]
X_train_POSTOP_STACKED['RF_PREOP_PROBA'] = rf_predict_probs[:, 1]
#X_train_POSTOP_STACKED['STS_predstro'] = predstro_train

In [425]:
X_train_POSTOP_STACKED.head()

,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl_AoXC,aortoccl_Balloon,asmtaodx_Normal,asmtaodx_Thickening,asmtaodx_atheroma<5mm,asmtaodx_atheroma>5mm,asmtaodx_mobile_plaques,cathbasassistind_BP_instability,cathbasassistind_CPB_wean,cathbasassistind_PCI_failure,cathbasassistwhen_Intra_OP,cathbasassistwhen_Post_OP,cathbasassistwhen_Pre_OP,cpbutil_Combination,cpbutil_Full_CPB,cperftyp_Antegrade,cperftyp_Both,cperftyp_Retrograde,ecmoind_Cadiac_Failure,ecmoind_Rescue_Salvage,ecmoind_Resp_Failure,ecmowhen_Intra_OP,ecmowhen_Post_OP,ecmowhen_Pre_OP,emergrsn_Anatomy,emergrsn_Angio_Accident,emergrsn_Ao_Dissect,emergrsn_Evolvg_MI,emergrsn_Hybrid,emergrsn_Infect_Device,emergrsn_Ischemia,emergrsn_Pulm_Edema,emergrsn_Shock_Circ_Supp,emergrsn_Shock_No_Circ_Supp,emergrsn_Syncope,emergrsn_Trauma,emergrsn_Valve_Dysf,iabpind_Angina,iabpind_CPB_Wean_Failure,iabpind_Hemodyn_Instab,iabpind_Procedure_Support,iabpind_Prophylactic,iabpwhen_Intra_OP,iabpwhen_Post_OP,iabpwhen_Pre_OP,unplproc_Yes_Complication,unplproc_Yes_Disease,urgntrsn_AMI,urgntrsn_Anatomy,urgntrsn_Angio_Accid,urgntrsn_Ao_Dissect,urgntrsn_CHF,urgntrsn_CP,urgntrsn_Hybrid,urgntrsn_IABP,urgntrsn_Infect_Dev,urgntrsn_PCI_Fail,urgntrsn_Rest_Angina,urgntrsn_Syncope,urgntrsn_Trauma,urgntrsn_USA,urgntrsn_Valve_Dysfunctn,LR_PREOP_PROBA,RF_PREOP_PROBA
0,-0.00693,-0.0698,-0.06628,-0.00465,-0.03571,5.70929,-0.19482,-2.84486,-0.76826,-0.00312,0.11526,-0.16015,2.00945,2.55393,0.53611,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.57155,0.21333
1,-0.00693,-0.0698,-0.06628,-0.00465,-0.03571,-0.18655,-0.19482,0.12646,-0.36199,0.26499,-0.80508,-0.52011,0.04493,0.03835,-0.42841,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.74170,0.29233
2,-0.00693,-0.0698,-0.06628,-0.00465,-0.03571,-0.18655,11.74946,-2.84486,0.04429,-0.67341,4.56355,0.91973,0.04493,0.03835,4.23865,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.48812,0.19102
3,-0.00693,-0.0698,-0.06628,-0.00465,-0.03571,-0.18655,4.58289,-2.84486,-0.76826,0.73420,-0.62612,0.55977,0.04493,0.03835,-0.77067,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.36227,0.07515
4,-0.00693,-0.0698,-0.06628,-0.00465,-0.03571,-0.18655,-0.19482,0.12646,1.26310,-0.00312,0.19195,-0.61010,0.04493,0.03835,-0.55287,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.

In [426]:
X_train_POSTOP_STACKED.shape

(23156, 113)

### Step 3a: Model and Evaluate `X_train_POSTOP_STACKED` Using `LogisticRegression()`

- instantiating a `LogisticRegression()` object

In [427]:
log_reg_clf = LogisticRegression()

- will use scoring metrics defined above in previous `GridSearchCV`
- defining hyperparameter grid

In [428]:
hyperparameters = {'penalty': ['l1'],
                   'C': [0.0001, 0.001, 0.01, 0.1, 1.0, 10.0],
                   'class_weight': ['balanced'],
                   'solver': ['liblinear'],
                   'random_state': [0]
                  }

- create `GridSearchCV` object

In [429]:
gs_stacked = GridSearchCV(log_reg_clf, #classifier
                          param_grid=hyperparameters, #param_grid
                          scoring=scoring_metrics, #loss functions
                          cv=5, #number of folds, default=`StratifiedKFold`
                          refit='roc_auc', # best_estimator_ will be based on this scoring metric
                          return_train_score=True,
                          n_jobs=-1,
                          verbose=5)

- fitting `GridSearchCV`

In [430]:
PREPOST_modelB = gs_stacked.fit(X_train_POSTOP_STACKED, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed:    1.8s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  3.8min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed:  3.8min finished


- examining results

In [431]:
gs_results = pd.DataFrame(PREPOST_modelB.cv_results_).sort_values(by=['mean_test_roc_auc'],
                                                                      ascending=False)

In [432]:
display_cols = ['param_C',
                'mean_test_roc_auc',
                'mean_train_roc_auc',
                'mean_test_f1',
                'mean_test_precision',
                'mean_test_recall']

In [433]:
gs_results_summary = gs_results.copy()[display_cols].reset_index(drop=True)

In [434]:
gs_results_summary

,param_C,mean_test_roc_auc,mean_train_roc_auc,mean_test_f1,mean_test_precision,mean_test_recall
0,0.01,0.72020,0.74828,0.05803,0.03054,0.57981
1,0.1,0.71517,0.77315,0.06067,0.03196,0.59780
2,1,0.68998,0.78258,0.05856,0.03085,0.57435
3,10,0.67868,0.78390,0.05759,0.03035,0.56254
4,0.001,0.62384,0.64266,0.04131,0.02138,0.60944
5,0.0001,0.50000,0.50000,0.00000,0.00000,0.00000


### Key Takeaways
- similar to `Modeling Scenario A` achieved better results than `STS` with the `POSTOP` features
- results were slightly below that of `Modeling Scenario A` on a `mean_test_roc_auc` basis
- extend this stacking concept to add additional vectors of probability of stroke from other models `RandomForest` and `STS`
- adding the `RandomForest` predicted probabilities improved results slightly, but the addition of `STS` on top of that did not improve the `mean_test_roc_auc`
- instead of modeling the stacked `POSTOP` data with `LogisticRegression` can try another model such as `RandomForest` or `SVM`
- in addition can also try using `PolynomialFeatures` transformation using `Scenario B` to see if any benefit from modeling the interaction of the `PREOP` predictions with the additional `POSTOP` features
- should examine the non-zero coeficients in `POSTOP` to get insight into what `POSTOP` features are useful in enhancing stroke prediction

## Step 3 ab - Just Preop Data RF

In [435]:
PREOP_RF_clf_pipe = Pipeline(steps=[
    ('get_PREOP_TREE_feature_matrix', PREOP_feature_matrix_TREE),
    ('Classifier', RandomForestClassifier())])

In [436]:
parameter_grid = {'Classifier__n_estimators': [315],
                  'Classifier__max_features': [0.1],
                  'Classifier__max_depth': [None],
                  'Classifier__min_samples_split': [50],
                  'Classifier__class_weight': ['balanced'],
                  'Classifier__random_state': [0]
                 }
scoring_metrics = ['accuracy',
                   'f1',
                   'f1_macro',
                   'f1_weighted',
                   'precision',
                   'precision_macro',
                   'precision_weighted',
                   'recall',
                   'recall_macro',
                   'recall_weighted',
                   'roc_auc']

In [437]:
gs = GridSearchCV(PREOP_RF_clf_pipe, #pipline
                  param_grid=parameter_grid, #param_grid
                  scoring=scoring_metrics, #loss functions
                  cv=5, #number of folds, default=`StratifiedKFold`
                  refit='roc_auc', # best_estimator_ will be based on this scoring metric
                  return_train_score=True,
                  n_jobs=-1,
                  verbose=5)

In [438]:
PRE_modelRF = gs.fit(X_train_all, y_train)

Fitting 5 folds for each of 1 candidates, totalling 5 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 out of   5 | elapsed:   32.0s remaining:   48.1s
[Parallel(n_jobs=-1)]: Done   5 out of   5 | elapsed:   50.3s finished


In [439]:
gs_results = pd.DataFrame(PRE_modelRF.cv_results_).sort_values(by=['mean_test_roc_auc'],
                                                                  ascending=False)
display_cols = ['param_Classifier__n_estimators',
                'mean_test_roc_auc',
                'mean_train_roc_auc',
                'mean_test_f1',
                'mean_test_precision',
                'mean_test_recall']
gs_results_summary = gs_results.copy()[display_cols].reset_index(drop=True)
gs_results_summary.rename(columns={'param_Classifier__n_estimators': 'n_estimators'},
                          inplace=True)

In [440]:
gs_results_summary

,n_estimators,mean_test_roc_auc,mean_train_roc_auc,mean_test_f1,mean_test_precision,mean_test_recall
0,315,0.68433,1.0,0.0,0.0,0.0


### Step 3b: Model and Evaluate Stacked Model Using `RandomForestClassifier`

#### Creating `POSTOP_feature_matrix_TREE` using `sklearn` `Pipelines` 

- can use similar coding pattern to assemble `POSTOP_feature_matrix_TREE` if needed
    - `POSTOP_numerical_features_pipeline` has been previously created above
    - would need to create `POSTOP_categorical_TREE_features_pipeline` -- THIS HAS NOT BEEN DONE YET
    - then use `FeatureUnion()` to assemble `POSTOP_feature_matrix_TREE`
    - similar to what was done above in using `Pipeline`s to create `X_train_PREOP`, `X_train_POSTOP` and corresponding `X_dev` and `X_test` 

In [441]:
POSTOP_categorical_TREE_features_pipeline = Pipeline(steps=[
    ('select', FeatureSelector(POSTOP_categorical_TREE_col_names)),
    ('convert_numpy', FunctionTransformer(convert_df_to_numpy, validate=False))
    ])

- using `FeatureUnion` combining using with already created `POSTOP_numerical_features_pipeline` to assemble `POSTOP_feature_matrix_TREE`

In [442]:
POSTOP_feature_matrix_TREE = FeatureUnion([('POSTOP_num_features', POSTOP_numerical_features_pipeline),
                                           ('POSTOP_cat_tree_features', POSTOP_categorical_TREE_features_pipeline)
                                          ])

#### NOTE - as we did above:

- `fit_transform` only on `X_train_all` for `X_train_POSTOP_TREE`
- `transform` on `X_dev_all` and `X_test_all` for `X_dev_POSTOP_TREE` and `X_test_POSTOP_TREE`
- assign compreleted `feature_matrix` to variable name if not using directly in a `Pipeline` with classifier step

#### Transforming `X_train_all` to get `X_train_POSTOP_TREE`

In [443]:
X_train_POSTOP_TREE = POSTOP_feature_matrix_TREE.fit_transform(X_train_all)

- validating

In [444]:
X_train_all.shape, X_train_POSTOP_TREE.shape, y_train.shape

((23156, 247), (23156, 62), (23156,))

- putting into a `DataFrame` for easy analysis

In [445]:
POSTOP_feature_TREE_col_names = POSTOP_numerical_col_names + POSTOP_categorical_TREE_col_names

In [446]:
len(POSTOP_feature_TREE_col_names), len(POSTOP_numerical_col_names), len(POSTOP_categorical_TREE_col_names)

(62, 15, 47)

In [447]:
X_train_POSTOP_TREE = pd.DataFrame(X_train_POSTOP_TREE,
                                   columns=POSTOP_feature_TREE_col_names)

In [448]:
X_train_POSTOP_TREE.head()

,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl,asmtaodx,cathbasassistind,cathbasassistwhen,cpbutil,cperftyp,ecmoind,ecmowhen,emergrsn,iabpind,iabpwhen,unplproc,urgntrsn
0,-0.00693,-0.0698,-0.06628,-0.00465,-0.03571,5.70929,-0.19482,-2.84486,-0.76826,-0.00312,0.11526,-0.16015,2.00945,2.55393,0.53611,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0
1,-0.00693,-0.0698,-0.06628,-0.00465,-0.03571,-0.18655,-0.19482,0.12646,-0.36199,0.26499,-0.80508,-0.52011,0.04493,0.03835,-0.42841,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,5.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.00693,-0.0698,-0.06628,-0.00465,-0.03571,-0.18655,11.74946,-2.84486,0.04429,-0.67341,4.56355,0.91973,0.04493,0.03835,4.23865,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,4.0,2.0,0.0,7.0
3,-0.00693,-0.0698,-0.06628,-0.00465,-0.03571,-0.18655,4.58289,-2.84486,-0.76826,0.73420,-0.62612,0.55977,0.04493,0.03835,-0.77067,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.00693,-0.0698,-0.06628,-0.00465,-0.03571,-0.18655,-0.19482,0.12646,1.26310,-0.00312,0.19195,-0.61010,0.04493,0.03835,-0.55287,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


#### Creating `X_train_POSTOP_TREE_STACKED`
- adding vectors of `predict_probas` from `PREOP` models and `STS` models to `POSTOP_TREE` feature matrix

In [449]:
X_train_POSTOP_TREE_STACKED = X_train_POSTOP_TREE.copy()

In [450]:
X_train_POSTOP_TREE_STACKED.shape, X_train_POSTOP_TREE.shape

((23156, 62), (23156, 62))

In [451]:
X_train_POSTOP_TREE_STACKED['LR_PREOP_PROBA'] = lr_predict_probs[:, 1]
X_train_POSTOP_TREE_STACKED['RF_PREOP_PROBA'] = rf_predict_probs[:, 1]
#X_train_POSTOP_TREE_STACKED['STS_predstro'] = predstro_train

In [452]:
X_train_POSTOP_TREE_STACKED.head()

,cperftime,cumulsatlft,cumulsatrt,dhcatm,ibdcryou,ibdffpu,ibdplatu,ibdrbcu,lwsthct,lwsttemp,perfustm,postcreat,prerso2lft,prerso2rt,xclamptm,canartstaort,canartstfem,canartstoth,canartstax,cathbasassist,ceroxused,circarr,cofirstind,concalc,cperfutil,IABP,ibldprod,imedeaca,imedtran,inoptee,ocarasd,ocarvsd,opocard,oponcard,unplao,unplav,unplmv,unplvad,vsmvpr,asmtascaa,cotafib,cotarrst,ecmo,mt30stat,opcab,opvalve,vadproc,vsavpr,vsmv,aortoccl,asmtaodx,cathbasassistind,cathbasassistwhen,cpbutil,cperftyp,ecmoind,ecmowhen,emergrsn,iabpind,iabpwhen,unplproc,urgntrsn,LR_PREOP_PROBA,RF_PREOP_PROBA
0,-0.00693,-0.0698,-0.06628,-0.00465,-0.03571,5.70929,-0.19482,-2.84486,-0.76826,-0.00312,0.11526,-0.16015,2.00945,2.55393,0.53611,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0,0.57155,0.21333
1,-0.00693,-0.0698,-0.06628,-0.00465,-0.03571,-0.18655,-0.19482,0.12646,-0.36199,0.26499,-0.80508,-0.52011,0.04493,0.03835,-0.42841,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,5.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.74170,0.29233
2,-0.00693,-0.0698,-0.06628,-0.00465,-0.03571,-0.18655,11.74946,-2.84486,0.04429,-0.67341,4.56355,0.91973,0.04493,0.03835,4.23865,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,3.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,4.0,2.0,0.0,7.0,0.48812,0.19102
3,-0.00693,-0.0698,-0.06628,-0.00465,-0.03571,-0.18655,4.58289,-2.84486,-0.76826,0.73420,-0.62612,0.55977,0.04493,0.03835,-0.77067,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.36227,0.07515
4,-0.00693,-0.0698,-0.06628,-0.00465,-0.03571,-0.18655,-0.19482,0.12646,1.26310,-0.00312,0.19195,-0.61010,0.04493,0.03835,-0.55287,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.49750,0.17895


In [453]:
X_train_POSTOP_TREE_STACKED.shape

(23156, 64)

- instantiating a `RandomForestClassifier()` object
- code pattern from line 169 - `GridSearchCV`

In [454]:
rf_clf = RandomForestClassifier(class_weight='balanced',
                                random_state=0)

In [455]:
hyper_params = {'n_estimators': [100, 200, 250, 300],          # default is `10`
                'max_features': ['auto', 0.10, 0.25, 0.50],    # default is square root of `n_features`    
                'max_depth': [2, 5, 10, None],                 # default is `None`
                'min_samples_split': [2, 5, 10]}               # default is `2`

In [456]:
gs = GridSearchCV(rf_clf, # estimator
                  hyper_params, # param_grid
                  scoring=scoring_metrics, # loss functions
                  cv=5, 
                  refit='roc_auc', # best_estimator_ will be based on this scoring metric
                  verbose=2,
                  return_train_score=True,
                  n_jobs=-1)

In [457]:
rf_gs_stacked_model = gs.fit(X_train_POSTOP_TREE_STACKED, 
                             y_train)

Fitting 5 folds for each of 192 candidates, totalling 960 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  5.4min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed: 64.5min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed: 115.5min
[Parallel(n_jobs=-1)]: Done 960 out of 960 | elapsed: 192.6min finished


- `GridSearchCV` takes approximately 90 minutes to run

#### Saving `rf_gs_stacked_model`

In [458]:
joblib.dump(rf_gs_stacked_model, 'rf_gs_stacked_model_cabg_nosts_12_3.sav')

['rf_gs_stacked_model_cabg_nosts_12_3.sav']

#### Loading saved `rf_gs_stacked_model`

In [459]:
#rf_gs_stacked_model = joblib.load('rf_gs_stacked_model_11_17.sav')

- analyzing results

In [460]:
rf_gs_stacked_model.best_params_

{'max_depth': 2,
 'max_features': 0.1,
 'min_samples_split': 10,
 'n_estimators': 300}

In [461]:
gs_results = pd.DataFrame(rf_gs_stacked_model.cv_results_)

In [462]:
gs_results = gs_results.sort_values(by=['mean_test_roc_auc'],
                                           ascending=False)

In [463]:
display_cols = ['params',
                'mean_test_roc_auc',
                'mean_train_roc_auc',
                'mean_test_f1',
                'mean_test_precision',
                'mean_test_recall']

In [464]:
gs_results_summary = gs_results.copy()[display_cols].reset_index(drop=True)

In [465]:
gs_results_summary.head()

,params,mean_test_roc_auc,mean_train_roc_auc,mean_test_f1,mean_test_precision,mean_test_recall
0,"{'max_depth': 2, 'max_features': 0.1, 'min_samples_split': 10, 'n_estimators': 300}",0.72279,0.76295,0.05533,0.02894,0.63016
1,"{'max_depth': 2, 'max_features': 0.1, 'min_samples_split': 2, 'n_estimators': 300}",0.72277,0.76289,0.05498,0.02875,0.62722
2,"{'max_depth': 2, 'max_features': 0.1, 'min_samples_split': 5, 'n_estimators': 300}",0.72276,0.76296,0.05533,0.02894,0.63016
3,"{'max_depth': 2, 'max_features': 0.1, 'min_samples_split': 2, 'n_estimators': 250}",0.72197,0.76175,0.05562,0.02908,0.63600
4,"{'max_depth': 2, 'max_features': 0.1, 'min_samples_split': 2, 'n_estimators': 200}",0.72195,0.76134,0.05450,0.02846,0.64504


#### Key Takeaways
- better than `STS` alone, but results slightly below that of `LogisticRegression()`

## Results on test set

In [466]:
def calcTestROC(model, X_test, y_test):
    pred_prob = model.predict_proba(X_test)[:,1]
    return roc_auc_score(y_test, pred_prob)

In [467]:
calcTestROC(PREPOST_modelA, X_test_all, y_test ) #log reg on pre+post data

0.7292955218897375

In [468]:
calcTestROC(PRE_modelA, X_test_all, y_test) #log reg on pre data

0.6627498852301658

In [469]:
calcTestROC(PRE_modelRF, X_test_all, y_test ) #RF on pre data

0.6917595258962481

In [470]:
#need to create X_test_POSTOP_STACKED
X_test_POSTOP_STACKED = X_test_POSTOP.copy()
X_test_POSTOP_STACKED['LR_PREOP_PROBA'] = PRE_modelA.predict_proba(X_test_all)[:, 1]
X_test_POSTOP_STACKED['RF_PREOP_PROBA'] = PRE_modelRF.predict_proba(X_test_all)[:, 1]

In [471]:
calcTestROC(PREPOST_modelB, X_test_POSTOP_STACKED, y_test ) #log reg on pre+post stacked

0.717434581194441

In [472]:
#need to create X_test_POSTOP_TREE_STACKED
X_test_POSTOP_TREE = POSTOP_feature_matrix_TREE.transform(X_test_all)
X_test_POSTOP_TREE = pd.DataFrame(X_test_POSTOP_TREE,
                                   columns=POSTOP_feature_TREE_col_names)
X_test_POSTOP_TREE_STACKED = X_test_POSTOP_TREE.copy()
X_test_POSTOP_TREE_STACKED['LR_PREOP_PROBA'] = PRE_modelA.predict_proba(X_test_all)[:, 1]
X_test_POSTOP_TREE_STACKED['RF_PREOP_PROBA'] = PRE_modelRF.predict_proba(X_test_all)[:, 1]

In [473]:
calcTestROC(rf_gs_stacked_model, X_test_POSTOP_TREE_STACKED, y_test ) #RF on pre+post stacked

0.7238105671716539

In [474]:
round(roc_auc_score(y_test,predstro_test),5)

0.70979

In [475]:
round(roc_auc_score(y_train,predstro_train),5)

0.69999

In [476]:
break

SyntaxError: 'break' outside loop (<ipython-input-476-6aaf1f276005>, line 4)

### `Polynomial` Transformation on Stacked Model
- to see if adding interaction terms between `PREOP` models predicted probabilities and `POSTOP` features can improve results

- instantiating `Pipeline` that will include `Polynomial` transformation of `X_train_POSTOP_STACKED`

In [ ]:
poly_lr_model_pipe = Pipeline(steps=[
    ('poly_features', PolynomialFeatures(degree=2, include_bias=False)),
    ('Classifier', LogisticRegression())
    ])

- defining a parameter grid for `GridSearchCV`

In [ ]:
parameter_grid = {'Classifier__penalty': ['l1'],
                  'Classifier__C': [0.0001, 0.001, 0.01, 0.10, 0.50, 1.0, 10.0],
                  'Classifier__class_weight': ['balanced'],
                  'Classifier__solver': ['liblinear'],
                  'Classifier__random_state': [0]
                 }

- create `GridSearchCV` object

In [ ]:
gs = GridSearchCV(poly_lr_model_pipe, # estimator
                  param_grid=parameter_grid, # param_grid
                  scoring=scoring_metrics, # loss functions
                  cv=5,
                  refit='roc_auc', # best_estimator_ will be based on this scoring metric
                  verbose=2,
                  return_train_score=True,
                  n_jobs=-1)

- fitting `GridSearchcV`

- `GridSearchCV` takes over 2 full hours to run

#### Saving `poly_log_reg_lasso`

In [ ]:
#joblib.dump(poly_log_reg_lasso, 'poly_log_reg_lasso_model_11_17.sav')

#### Loading `poly_log_reg_lasso`

In [ ]:
poly_log_reg_lasso = joblib.load('poly_log_reg_lasso_model_11_17.sav')

- examining results

In [ ]:
poly_log_reg_lasso.best_params_

In [ ]:
gs_results = pd.DataFrame(poly_log_reg_lasso.cv_results_).sort_values(by=['mean_test_roc_auc'],
                                                                      ascending=False)

In [ ]:
display_cols = ['params',
                'mean_test_roc_auc',
                'mean_train_roc_auc',
                'mean_test_f1',
                'mean_test_precision',
                'mean_test_recall']

In [ ]:
gs_results_summary = gs_results.copy()[display_cols].reset_index(drop=True)

In [ ]:
gs_results_summary

#### Key Takeways
- `PolynomialTransformation` actually had worse results than non-transformed `LogisticRegression`
- Results were in-line with `STS` model

## CODE BELOW IS USEFUL FOR DIAGNOSTIC CHARTS ON MODEL SCORING

### Need to modify and adapt for code above given changes in variable names

In [ ]:
break

# ---------------------------------------------------------------------------------------------

## Visualizing the Effect of `LogisticRegression()` Hyperparameter Values
- See Albon `11.13` pages 205-207
- range of values for `C` - inverse of regularization strength

In [ ]:
param_values = np.linspace(0.001, 0.05, 25)

- calculating `roc_auc` on training and test set using `param_range` for `C`

In [ ]:
train_scores, test_scores = validation_curve(LogisticRegression(penalty='l1',
                                                                class_weight='balanced',
                                                                random_state=0,
                                                                solver='liblinear'), #classifier
                                             X_train, #feature matrix
                                             y_train, #target vector
                                             param_name='C', #hyperparameter to examine,
                                             param_range=param_values, #range of hyperparameter values
                                             cv=5, #number of folds, default=`StratifiedKFold`
                                             scoring='roc_auc', #scoring metric
                                             n_jobs=-1, #use all computer cores
                                             verbose=5) 

- calculate `mean` and `standard deviation` for `training` set scores

In [ ]:
train_mean = np.mean(train_scores, axis=1)
train_std = np.std(train_scores, axis=1)

- calculate `mean` and `standard deviation` for `test` set scores

In [ ]:
test_mean = np.mean(test_scores, axis=1)
test_std = np.std(test_scores, axis=1)

- Plotting

In [ ]:
plt.rcParams["figure.figsize"] = (20, 10)
plt.style.use('fivethirtyeight')

# plot mean accuracy scores for training and test sets
plt.plot(param_values,
         train_mean,
         label='Training Score',
         color='midnightblue')

plt.plot(param_values,
         test_mean,
         label='Cross Validation Score',
         color='darkviolet')

# plot accuracy bands for training and test sets
plt.fill_between(param_values, 
                 train_mean - train_std,
                 train_mean + train_std,
                 color='lightsteelblue')

plt.fill_between(param_values,
                 test_mean - test_std,
                 test_mean + test_std,
                 color='thistle')

# create plot
plt.title('Validation Curve for Logistic Regression with l1 Regularization')
plt.xlabel('C')
plt.ylabel('AUCROC')
plt.legend(loc='lower right')
plt.show()       

In [ ]:
validation_summary = pd.DataFrame({'C': param_values,
                                   'mean_test_roc_auc': test_mean,
                                   'mean_train_roc_auc': train_mean})

In [ ]:
validation_summary[validation_summary['mean_test_roc_auc'] == np.max(validation_summary['mean_test_roc_auc'])]

#### How does this tuned model compare to the baseline `STS` Model

In [ ]:
round(roc_auc_score(y_train, predstro_train), 5)

#### Key Takeaways
- Fine tuning of hyperparmeter `C` results in `C=0.0275`
- Results in-line (but still lower) than `STS` Model

## Precision-Recall Curve for Tuned Logistic Regression Model
- `Hands On Machine Learning with Scikit-Learn & Tensorflow` by Aurelien Geron pages 83-93
#### How `cross_val_predict` works:
- `cross_val_predict` returns for each element in the input, the prediction that was obtained for that element when it was in the `test` set

- generating vector of probabilities of stroke for cross validated training set

In [ ]:
lr_predict_probs = cross_val_predict(LogisticRegression(penalty='l1',
                                                        C=0.0275,
                                                        class_weight='balanced',
                                                        random_state=0,
                                                        solver='liblinear'),
                                     X_train,
                                     y_train,
                                     cv=5,
                                     method='predict_proba',
                                     n_jobs=-1,
                                     verbose=5)

- using `lr_predict_probs` can compute precision and recall for all possible thresholds using the `precision_recall_curve` function
- returns an `numpy.ndarray` with `shape` `(n, 2)` as it returns a probability per class `(0, 1)`
- since we are interested in the `1` class, need to carve our the second column of the `numpy.ndarray`
- add `[:,1]`

In [ ]:
type(lr_predict_probs)

In [ ]:
lr_predict_probs.shape

In [ ]:
lr_predict_probs[0:5]

In [ ]:
lr_predict_probs[0:5, 1]

#### Generating `precision_recall_curve`

In [ ]:
precisions, recalls, thresholds = precision_recall_curve(y_train,
                                                         lr_predict_probs[:, 1])

- `precision` values such that element `i` is the the `precision` of predictions with `score>=threholds[i]` and the last element is `1`

In [ ]:
precisions[0:5] # first 5 elements of the numpy array

In [ ]:
precisions[-5:] # last 5 elements of the numpy array

- decreasing `recall` values such that element `i` is the the `precision` of predictions with `score>=threholds[i]` and the last element is `0`

In [ ]:
recalls[0:5]

In [ ]:
recalls[-5:]

- increasing `thresholds` on the decision function used to compute `precision` and `recall`

In [ ]:
thresholds[0:5]

In [ ]:
thresholds[-5:]

### Plotting `precision` and `recall` as functions of `threshold`

In [ ]:
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
    plt.plot(thresholds, precisions[:-1], 'b--', label='Precision')
    plt.plot(thresholds, recalls[:-1], 'g-', label='Recall')
    plt.title('Precision and Recall versus Threshold')
    plt.xlabel('Threshold')
    plt.legend(loc='upper right')
    plt.ylim([0, 1])

### Precision and Recall versus Threshold for Logistic Regression Model

In [ ]:
plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
plt.show()

- helper function

In [ ]:
def find_nearest(array, value):
    array = np.asarray(array)
    idx = (np.abs(array - value)).argmin()
    return array[idx], idx

- examining specific `threshold`, `Recall` and `Precision` trade-offs
- suppose you wanted a `recall` of `0.80`, what `threshold` will you need and what will the `precision` be?

In [ ]:
find_nearest(recalls, 0.80)

In [ ]:
print('{0:15} {1:5f}'.format('Recall:', recalls[14806]))
print('{0:15} {1:5f}'.format('Precision:', precisions[14806]))
print('{0:15} {1:5f}'.format('Threshold:', thresholds[14806]))

- see `Geron` page 90 for additional details

### Precision and Recall versus Threshold for STS Model

In [ ]:
sts_precisions, sts_recalls, sts_thresholds = precision_recall_curve(y_train,
                                                                     predstro_train)

In [ ]:
plot_precision_recall_vs_threshold(sts_precisions, sts_recalls, sts_thresholds)
plt.show()

## Precision-Recall Curves
- `Introduction to Machine Learning with Python` by Andreas C. Muller and Sarah Guido pages 289-296
- the closer a `precision-recall curve` stays to the upper-right corner, the better classifier

In [ ]:
def plot_precision_recall_curve(precisions, recalls):
    plt.plot(precisions,
             recalls,
             color='blue',
             label='Precision-Recall Curve')
    plt.title('Precision-Recall Curve')
    plt.ylabel('Recall')
    plt.xlabel('Precision')
    plt.legend(loc='upper right')

### `Precision-Recall Curve` for `LogisticRegression` Model

In [ ]:
plot_precision_recall_curve(precisions, recalls)
plt.show()

#### Area Under the `Precision-Recall Curve` for `LogisticRegression` Model

In [ ]:
round(auc(recalls, precisions), 5)

### `Precision-Recall Curve` for `STS` Model

In [ ]:
plot_precision_recall_curve(sts_precisions, sts_recalls)
plt.show()

#### Area Under the `Precision-Recall Curve` for `STS` Model

In [ ]:
round(auc(sts_recalls, sts_precisions), 5)

#### Key Takeaways
- `STS` model is superior to `LogisticRegression` model looking at `Precision-Recall Curves` and `PR AUC`

### Comparing `Logistic Regression` and `STS` Model `Precision-Recall` Curves

In [ ]:
plt.rcParams["figure.figsize"] = (20, 10)
plt.style.use('fivethirtyeight')

# plotting `LogisticRegression` Precision-Recall Curve
plt.plot(precisions,
         recalls,
         'b--',
         label='Logistic Regression Model')

# plotting `STS Model` Precision-Recall Curve
plt.plot(sts_precisions,
         sts_recalls,
         'g-',
         label='STS Model')

plt.title('Logistic Regression, STS Model Precision-Recall Curves')
plt.ylabel('Recall')
plt.xlabel('Precision')
plt.legend(loc='upper right')
plt.xlim([0, 0.10]) # cut off values > 0.10 because they were essentially zero
plt.show()

#### Key Takeaways
- plot clearly shows `STS` model outperperformance over `LogisticRegression`
- can use basic code here to plot `Precision-Recall Curves` from other models on the same plot for comparison

## Receiver Operating Characteristic (`ROC`) Curves
- Albon `11.5` pages 189-192
- a classifier that predicts every observation correctly would look like the solid `gray` line in the plotted `ROC` curves below
- the solid `gray` line goes straight up to the top immediately
- a classifier that predicts at random will appear as the diagonal line
- the better the model, the closer it is to the solid `gray` line
- the `ROC` curve represents the respective `TPR` and `FPR` for every probability threshold

- creating `true` and `false` probabilities

In [ ]:
false_positive_rate, true_positive_rate, threshold = roc_curve(y_train,
                                                               lr_predict_probs[:, 1])

- `plot_roc_curve` function

In [ ]:
def plot_roc_curve(fpr, tpr):
    plt.title('Receiver Operating Curve (ROC)')
    plt.plot(fpr, tpr)
    plt.plot([0, 1], ls='--')
    plt.plot([0, 0],
             [1, 0],
             c='0.7'),
    plt.plot([1, 1], c='0.7')
    plt.ylabel('True Positive Rate (Recall)')
    plt.xlabel('False Positive Rate')

#### `LogisiticRegression` Model `ROC` Curve

In [ ]:
plt.rcParams["figure.figsize"] = (20, 10)
plt.style.use('fivethirtyeight')
plot_roc_curve(false_positive_rate, true_positive_rate)
plt.show()

In [ ]:
round(roc_auc_score(y_train, lr_predict_probs[:, 1]), 5)

- suppose you want `recall` in a certain range - what is the `false positive rate` you would have to accept?

In [ ]:
lr_roc_curve = pd.DataFrame({'Recall': true_positive_rate,
                             'FPR': false_positive_rate})

- assume you want `recall` around `0.80`

In [ ]:
lr_roc_curve[lr_roc_curve['Recall'].between(0.799, 0.801)]

- could have also used the `find_nearest` helper function above

In [ ]:
lr_roc_curve[lr_roc_curve['Recall'] == (find_nearest(lr_roc_curve['Recall'], 0.80)[0])]

- suppose you want to know `recall` and `FPR` for a given threshold

In [ ]:
np.where(threshold == 0.10)[0]

In [ ]:
print('{0:15} {1:5f}'.format('Threshold:', threshold[11]))
print('{0:15} {1:5f}'.format('Recall:', true_positive_rate[11]))
print('{0:15} {1:5f}'.format('FPR:', false_positive_rate[11]))

#### `STS` Model `ROC` Curve

In [ ]:
sts_fpr, sts_tpr, threshold = roc_curve(y_train,
                                        predstro_train)

In [ ]:
plt.rcParams["figure.figsize"] = (20, 10)
plt.style.use('fivethirtyeight')
plot_roc_curve(sts_fpr, sts_tpr)
plt.show()

In [ ]:
round(roc_auc_score(y_train, predstro_train), 5)

#### Key Takeaways
- `STS` Model slightly outperforms `LogisticRegression`